In [ ]:
import import_ipynb
import utils
import models
utils.hide_toggle('Imports 1')

In [ ]:
from IPython import display
import torch
from sklearn.manifold import TSNE
from matplotlib import pyplot as plt
from l2lutils import KShotLoader
from IPython import display
import torch.nn as nn
utils.hide_toggle('Imports 2')

In [ ]:
class CNP(nn.Module):
    def __init__(self,n_features=1,dims=[32,32],n_ways=5,n_classes=2,lr=1e-4):
        super(CNP,self).__init__()
        self.n_features = n_features
        self.n_classes = n_classes
        self.n_ways = n_ways
        dimL1 = [n_features]+dims
        dimL2=[n_features+n_classes*dims[-1]]+dims+[n_classes]
        self.mlp1 = models.MLP(dims=dimL1,task='embedding')
        self.mlp2 = models.MLP(dims=dimL2)
        self.optimizer=torch.optim.Adam(self.parameters(),lr=lr)
    def adapt(self,X,y):
        R = self.mlp1(X)
        m = torch.eye(self.n_classes)[y].transpose(0,1)/self.n_ways
        r = (m@R).flatten().unsqueeze(0)
        #r = (R.sum(dim=0)/X.shape[0]).unsqueeze(0)
        #return r
        return r,m,R
    def forward(self,Y,r):
        rr = r.repeat(Y.shape[0],1)
        p = self.mlp2(torch.cat((Y,rr),dim=1))
        return p
utils.hide_toggle('Class CNP')